# Реальный кейс с использованием t-тест для зависимых выборок

Я работаю Java-разработчиком в логистической компании. Нахожусь в отделе, занимающемся приложениями для банков. Мне приходится решать самые различные проблемы, например, иногда приходится исправлять базу данных заказчиков, не видя её, или исследовать ситуации, когда нагрузка на БД очень большая.

В таких случаях, чаще всего приходится оптимизировать скрипты в коде. Я оптимизирую скрипт и специальными методами анализирую его нагрузку. Когда я вижу, что нагрузка достаточно маленькая, я отправляю её на проверку заказчику. 

Тестовые базы и базы прода отличаются тем, что в базе прода обычно огромное количество данных, и скрипт может там отработать по-другому. Поэтому в ответ на скрипт, инженеры поддержки банка отправляют мне ответ, помогло ли это базе на проде или нет.

Недавно я попробовала метод t-теста для зависимых выборок, чтобы объективно оценить эффективность оптимизации скриптов с помощью статистических методов.

Рассмотрим его подробнее:

Импортируем нужные библиотеки

In [3]:
import numpy as np
import matplotlib.pyplot as plt